<img style="float: right;" src="imagenes/logo-bvmc.png" width="180px">
<img style="float: right;margin-right:10px;" src="imagenes/logo-ua.jpg" width="200px">
<img style="float: right;margin-right:10px;margin-top: -10px;" src="imagenes/logo-hdh.png" width="120px">

La <a href="https://data.cervantesvirtual.com">Biblioteca Virtual Miguel de Cervantes</a> publicó su catálogo como datos abiertos y enlazados (en inglés, Linked Open Data) utilizando como vocabulario principal <a href="http://www.rdaregistry.info/">Resource, Description and Access</a>.

Este Jupyter Notebook muestra cómo acceder al catálogo a través del lenguaje de consulta <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a>.

En primer lugar, incluimos las librerías de software necesarias para poder realizar una llamada al punto de acceso público SPARQL de la Biblioteca Virtual Miguel de Cervantes <a href="https://data.cervantesvirtual.com/sparql">https://data.cervantesvirtual.com/sparql</a>. También especificamos el grafo por defecto a utilizar.

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://data.cervantesvirtual.com/sparql"
)
sparql.setReturnFormat(JSON)
sparql.addDefaultGraph("http://localhost:8890/DAV/bvmc")

A continuación, especificamos la sentencia SPARQL para recuperar el título y autor de la obra El ingenioso hidalgo Don Quijote de la Mancha.

In [2]:
sparql.setQuery("""
    PREFIX rdac: <http://rdaregistry.info/Elements/c/>
    PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
    PREFIX rdaa: <http://rdaregistry.info/Elements/a/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT DISTINCT ?titulo ?autor ?nombre ?materia
    WHERE {
        <https://data.cervantesvirtual.com/work/2904> a rdac:Work .
        <https://data.cervantesvirtual.com/work/2904> rdaw:titleOfTheWork ?titulo .
        <https://data.cervantesvirtual.com/work/2904> dc:subject ?materia .
        <https://data.cervantesvirtual.com/work/2904> rdaw:author ?autor .
        ?autor rdaa:nameOfThePerson ?nombre
    }
    LIMIT 10
    """
)

Finalmente, mostramos el resultado por pantalla:

In [3]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Título: ', r['titulo']['value'])
        print('Materia: ', r['materia']['value'])
        print('Autor: ', r['autor']['value'])
        print('Nombre: ', r['nombre']['value'])
except Exception as e:
    print(e)

Título:  El ingenioso hidalgo Don Quijote de la Mancha
Materia:  Novela española -- Siglo 17º
Autor:  https://data.cervantesvirtual.com/person/40
Nombre:  Cervantes Saavedra, Miguel de


A continuación, vamos a recuperar los personajes de la obra que se encuentran disponibles en Wikidata. Para ello utilizamos la propiedad wdt:P3976 que identifica obras en la Biblioteca Virtual Miguel de Cervantes.

In [4]:
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
    PREFIX rdac: <http://rdaregistry.info/Elements/c/>
    PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
    PREFIX rdaa: <http://rdaregistry.info/Elements/a/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT DISTINCT ?personaje ?personajeLabel
    WHERE {
        ?work wdt:P3976 "2904".
        ?work wdt:P674 ?personaje . 
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
    }
    LIMIT 50
    """
)

Y de la misma forma, mostramos los resultados:

In [5]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Personaje: ', r['personaje']['value'])
        print('Personaje: ', r['personajeLabel']['value'])
except Exception as e:
    print(e)

Personaje:  http://www.wikidata.org/entity/Q5129635
Personaje:  Clavileño
Personaje:  http://www.wikidata.org/entity/Q6120178
Personaje:  Sansón Carrasco
Personaje:  http://www.wikidata.org/entity/Q6072772
Personaje:  Pero Pérez
Personaje:  http://www.wikidata.org/entity/Q5563337
Personaje:  Ginés de Pasamonte
Personaje:  http://www.wikidata.org/entity/Q6113413
Personaje:  el rucio
Personaje:  http://www.wikidata.org/entity/Q23807432
Personaje:  Maese Nicolás
Personaje:  http://www.wikidata.org/entity/Q52733938
Personaje:  Marcela
Personaje:  http://www.wikidata.org/entity/Q56419457
Personaje:  Don Fernando
Personaje:  http://www.wikidata.org/entity/Q21401597
Personaje:  Maritornes
Personaje:  http://www.wikidata.org/entity/Q630823
Personaje:  Sancho Panza
Personaje:  http://www.wikidata.org/entity/Q1918252
Personaje:  Rocinante
Personaje:  http://www.wikidata.org/entity/Q52734852
Personaje:  Dorotea
Personaje:  http://www.wikidata.org/entity/Q56480586
Personaje:  Anselmo
Personaje:  h

También podemos mostrar las imágenes de los personajes:

In [8]:
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
    PREFIX rdac: <http://rdaregistry.info/Elements/c/>
    PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
    PREFIX rdaa: <http://rdaregistry.info/Elements/a/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT DISTINCT ?imagen ?personajeLabel
    WHERE {
        ?work wdt:P3976 "2904".
        ?work wdt:P674 ?personaje . 
        ?personaje wdt:P18 ?imagen .
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
    }
    LIMIT 5
    """
)

In [11]:
from IPython.display import display, HTML

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Personaje: ', r['personajeLabel']['value'])
        print('Imagen: ', r['imagen']['value'])
        display(HTML("<img width='100px' src='"+ r['imagen']['value'] +"'>"))
except Exception as e:
    print(e)

Personaje:  Sancho Panza
Imagen:  http://commons.wikimedia.org/wiki/Special:FilePath/1879%2C%20El%20ingenioso%20hidalgo%20D.%20Quijote%20de%20la%20Mancha%2C%20Sancho%20Panza%2C%20Mestres%20%28cropped%29.jpg


Personaje:  Anselmo
Imagen:  http://commons.wikimedia.org/wiki/Special:FilePath/202%20084%20quijote%20cap33%20%28cropped%29%20Anselmo.jpg


Personaje:  Rocinante
Imagen:  http://commons.wikimedia.org/wiki/Special:FilePath/CavalloChishotte.jpg


Personaje:  Maritornes
Imagen:  http://commons.wikimedia.org/wiki/Special:FilePath/Don%20Quijote%2C%20Luis%20Tasso%2C%20%281894%3F%29%20%22El%20lecho%2C%20que%20era%20un%20poco%20endeble...d%C3%ADo%20consigo%20en%20el%20suelo...%22%20%285789341875%29%20%28cropped%29%20Maritornes.jpg


Personaje:  Dorotea
Imagen:  http://commons.wikimedia.org/wiki/Special:FilePath/1905-05-27%2C%20Blanco%20y%20Negro%2C%20Dorotea%2C%20Cecilio%20Pla%20%28cropped%29.jpg
